In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os, sys

In [4]:
sys.path.append("..")

In [5]:
import nlp_tools

In [68]:
text = "THE PT PRESENTS TODAY WITH DIABETES AND FOR R/O OF PNEUMONIA."

In [69]:
nlp = nlp_tools.build_nlp(disable="ner")

In [70]:
nlp.pipe_names

['tagger', 'parser', 'target_matcher', 'context', 'postprocessor']

# I. Preprocessor

In [71]:
preprocessor = nlp.tokenizer

In [72]:
preprocessor

In [73]:
import re

In [74]:
preprocess_rules = [
    lambda x: x.lower(),
    nlp_tools.PreprocessingRule(re.compile("pt"), "patient"),
    nlp_tools.PreprocessingRule(re.compile("r/o"), "rule out"),
]

In [75]:
preprocessor.add(preprocess_rules)

In [76]:
nlp(text)

the patient presents today with diabetes and for rule out of pneumonia.

# II. TargetMatcher

In [77]:
target_matcher = nlp.get_pipe("target_matcher")

In [78]:
target_matcher

In [79]:
target_rules = [
    nlp_tools.TargetRule("diabetes", "PROBLEM"),
    nlp_tools.TargetRule("pneumonia", "PROBLEM")
]

In [80]:
target_matcher.add(target_rules)

In [81]:
doc = nlp(text)

In [82]:
doc.ents

(diabetes, pneumonia)

# III. ConText

In [83]:
from cycontext.viz import visualize_dep, visualize_ent

In [84]:
visualize_ent(doc)

In [85]:
visualize_dep(doc)

In [86]:
for ent in doc.ents:
    print(ent._.is_uncertain)

False
True


# IV. Postprocessor

In [87]:
postprocessor = nlp.get_pipe("postprocessor")

In [89]:
from nlp_tools import postprocessing_functions

In [95]:
postprocess_rules = [
    nlp_tools.PostprocessingRule(
        patterns=[
            nlp_tools.PostprocessingPattern(lambda x: x._.is_uncertain),
        ],
        action=postprocessing_functions.remove_ent
    ),
    
]

In [96]:
postprocessor.add(postprocess_rules)

In [97]:
doc = nlp(text)

In [98]:
doc.ents

(diabetes,)

In [99]:
visualize_ent(doc)